In [ ]:
import numpy as np

class NeuralNetwork:
    def __init__(self, num_layers, neurons):
        self.num_layers = num_layers
        self.neurons = neurons
        self.weights = [np.random.randn(neurons[i], neurons[i+1]) / np.sqrt(neurons[i]) for i in range(num_layers - 1)]
        self.biases = [np.zeros((1, neurons)) for neurons in neurons[1:]]  #Initialize biases with zeros for each layer

    def relu(self, Z):  #Relu
        return np.maximum(0, Z)

    def svm_loss(self, scores, y):
        num_samples = scores.shape[0]
        correct_scores = scores[np.arange(num_samples), y]
        margins = np.maximum(0, scores - correct_scores[:, np.newaxis] + 1.0)
        margins[np.arange(num_samples), y] = 0
        loss = np.sum(margins) / num_samples
        margins[margins > 0] = 1  # Set margins as upper derivatives
        margins[np.arange(num_samples), y] = -np.sum(margins, axis=1)  # Subtract the number of positive margins for correct classes
        return loss, margins

    def forward_pass(self, X):
        activations = [X]
        for i in range(self.num_layers - 1):
            Z = np.dot(activations[i], self.weights[i]) + self.biases[i]
            A = self.relu(Z)
            activations.append(A)
        return activations

    def train(self, X_train, y_train, learning_rate=0.01, num_epochs=1000):
        for epoch in range(num_epochs):
            activations = self.forward_pass(X_train)
            scores = activations[-1]
            loss, margins = self.svm_loss(scores, y_train)

            # Backpropagation
            f = margins  # Use margins as upper derivatives
            num_samples = X_train.shape[0]
            f = f / num_samples  # Normalize gradients
            for i in range(self.num_layers - 2, -1, -1):
                dZ = f
                dZ[activations[i + 1] <= 0] = 0  # ReLU derivative
                dW = np.dot(activations[i].T, dZ)
                db = np.sum(dZ, axis=0, keepdims=True)
                f = np.dot(dZ, self.weights[i].T)
                self.weights[i] -= learning_rate * dW
                self.biases[i] -= learning_rate * db

            if epoch % 100 == 0:
                print(f"Epoch {epoch}, Loss: {loss}")

    def predict(self, X_test):  #Predicts the classess
        activations = self.forward_pass(X_test)
        scores = activations[-1]
        predicted_class = np.argmax(scores, axis=1)
        return predicted_class


In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Define the neural network architecture
num_layers = 3
neurons = [X_train.shape[1], 10, len(set(y_train))]  # Input layer size, hidden layer size, output layer size

# Create and train the neural network
nn = NeuralNetwork(num_layers, neurons)
nn.train(X_train, y_train, learning_rate=0.01, num_epochs=1000)

# Make predictions on the training and testing sets
y_train_pred = nn.predict(X_train)
y_test_pred = nn.predict(X_test)

# Calculate accuracy
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

print("Training Accuracy:", train_accuracy)
print("Testing Accuracy:", test_accuracy)


Epoch 0, Loss: 1.0610052529442877
Epoch 100, Loss: 0.41621483237543483
Epoch 200, Loss: 0.34392124292943854
Epoch 300, Loss: 0.28441504663205414
Epoch 400, Loss: 0.22515881799314882
Epoch 500, Loss: 0.18506093246863775
Epoch 600, Loss: 0.16323362636873043
Epoch 700, Loss: 0.14593348669420841
Epoch 800, Loss: 0.13530741048928016
Epoch 900, Loss: 0.12521844398701518
Training Accuracy: 0.9523809523809523
Testing Accuracy: 0.9777777777777777
